<a href="https://colab.research.google.com/github/hyona-yu/dacon5/blob/master/%EC%83%88%EB%A7%88%EC%9D%8C%EC%83%88%EB%9C%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

In [0]:
train_csv = pd.read_csv('./train.csv')
test_csv = pd.read_csv('./test.csv')

In [0]:
columns = train_csv.columns
label_columns = ['hhb','hbo2','ca','na']

In [0]:
train_csv = train_csv.interpolate(method = 'linear', axis = 1)
test_csv = test_csv.interpolate(method=  'linear', axis =1)

In [0]:
sns.set()
for c in columns[1:]:
    fig,ax = plt.subplots(1,1, figsize= (20,4))
    sns.distplot(train_csv[c])

In [0]:
new_test_csv = test_csv.drop(columns= ["id"],axis = 1)
new_train_csv = train_csv.drop(columns = label_columns, axis = 1).drop(columns = ['id'], axis = 1)

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler=  StandardScaler()
Mscaler = MinMaxScaler()
new_train_csv[columns[37:72]] = Mscaler.fit_transform(new_train_csv[columns[37:72]])
new_test_csv[columns[37:72]] = Mscaler.transform(new_test_csv[columns[37:72]])
new_train_csv[columns[2:37]] = scaler.fit_transform(new_train_csv[columns[2:37]])
new_test_csv[columns[2:37]] = scaler.transform(new_test_csv[columns[2:37]])
# new_train_csv[columns[2:72]]=scaler.fit_transform(new_train_csv[columns[2:72]]) #이거 별차이는 없음
# new_test_csv[columns[2:72]] =scaler.transform(new_test_csv[columns[2:72]])

In [0]:
# from sklearn.decomposition import PCA #별로
# pca = PCA(n_components= 1)
# new_train_csv[columns[37:72]] = pca.inverse_transform(pca.fit_transform(new_train_csv[columns[37:72]]))
# new_test_csv[columns[37:72]] = pca.inverse_transform(pca.transform(new_test_csv[columns[37:72]]))

In [0]:
train_D = new_train_csv
test_D = new_test_csv
train_L = train_csv.loc[:,"hhb":"na"]

In [154]:
train_D.rho.describe()

count    10000.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: rho, dtype: float64

In [128]:
type(train_D)

pandas.core.frame.DataFrame

#Train

In [0]:
# from sklearn.model_selection import KFold, StratifiedKFold
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import mean_absolute_error
# import xgboost as xgb
# import tqdm
# from sklearn.ensemble import RandomForestRegressor
# RF= RandomForestRegressor()
# kfold = KFold(shuffle=True, random_state=42)
# error = 0
# result_pred = np.zeros((len(test_csv),4))
# for train, test in (kfold.split(train_D, train_L)):
#     kfold_D , kfold_L = train_D.iloc[train,:], train_L.iloc[train,:] #train_D[train,], train_L.iloc[train,:]#
#     test_kfold_D, test_kfold_L = train_D.iloc[test,:], train_L.iloc[test,]#train_D[test], train_L.iloc[test,:]
    
#     RF.fit(kfold_D,  kfold_L)
#     pred= RF.predict(test_kfold_D)

#     error +=mean_absolute_error(pred, test_kfold_L)
#     print(mean_absolute_error(pred, test_kfold_L))
#     result_pred += RF.predict(test_D.iloc[:,:])/5
# print(error/5)

#LGBM

In [0]:
def train_model(x_data, y_data, k=5):
    models = []
    
    k_fold = KFold(n_splits=k, shuffle=True, random_state=123)
    
    for train_idx, val_idx in k_fold.split(x_data):
  
        x_train, y_train = x_data.iloc[train_idx,:], y_data[train_idx]
        x_val, y_val = x_data.iloc[val_idx,:], y_data[val_idx]

        d_train = lgb.Dataset(data = np.array(x_train), label =(y_train))
 
        d_val = lgb.Dataset(data = np.array(x_val), label = np.array(y_val))

        wlist = {}
       
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'seed':777,
            'feature_fraction':0.8,
            'bagging_fraction': 0.8,
            }

        model = lgb.train(params=params,
                          train_set=d_train,
                          valid_sets = [d_train, d_val],
                          valid_names = ['train', 'eval'],
                          num_boost_round=500,
                          verbose_eval=100,
                          evals_result=wlist)
        
        models.append(model)
    
    return models

In [0]:
# models = {}
# #이렇게 하는 거 의미없음..!
# models['hhb'] = train_model(train_D[columns[35:72]], train_L['hhb'])
# models['hbo2'] = train_model(train_D.drop(columns=columns[35:45],axis= 1), train_L['hbo2'])
# models['ca'] = train_model(train_D[columns[35:72]], train_L['ca'])
# models['na'] = train_model(train_D[columns[40:68]], train_L['na'])

In [144]:
models = {}
for label in label_columns:
    print('train column : ', label)
    models[label] = train_model(train_D, train_L[label])
    print('\n\n')

train column :  hhb
[100]	train's l1: 0.620281	eval's l1: 0.899414
[200]	train's l1: 0.446178	eval's l1: 0.856701
[300]	train's l1: 0.340455	eval's l1: 0.83917
[400]	train's l1: 0.26484	eval's l1: 0.827728
[500]	train's l1: 0.207118	eval's l1: 0.824058
[100]	train's l1: 0.632114	eval's l1: 0.863119
[200]	train's l1: 0.454729	eval's l1: 0.830328
[300]	train's l1: 0.344046	eval's l1: 0.817413
[400]	train's l1: 0.265109	eval's l1: 0.809658
[500]	train's l1: 0.207531	eval's l1: 0.805839
[100]	train's l1: 0.629106	eval's l1: 0.875946
[200]	train's l1: 0.452057	eval's l1: 0.831416
[300]	train's l1: 0.343431	eval's l1: 0.809658
[400]	train's l1: 0.266494	eval's l1: 0.799665
[500]	train's l1: 0.211292	eval's l1: 0.795363
[100]	train's l1: 0.62644	eval's l1: 0.885909
[200]	train's l1: 0.45369	eval's l1: 0.852092
[300]	train's l1: 0.340519	eval's l1: 0.832281
[400]	train's l1: 0.261789	eval's l1: 0.822826
[500]	train's l1: 0.205538	eval's l1: 0.817903
[100]	train's l1: 0.626918	eval's l1: 0.8907

In [0]:
sample = pd.read_csv('sample_submission.csv')
for col in models:
    preds = []
    for model in models[col]:
        preds.append(model.predict(test_D))
    pred = np.mean(preds, axis=0)

    sample[col] = pred

In [0]:
sample.to_csv('lgb_includerho_scaler_srcdst_all.csv', index = False)

In [147]:
pd.read_csv('lgb_includerho_scaler_srcdst_all.csv')

,id,hhb,hbo2,ca,na
0,10000,8.740354,4.468437,9.541955,3.755097
1,10001,6.264494,3.520176,8.437601,2.952617
2,10002,9.748657,5.093946,9.972243,3.865701
3,10003,8.435655,4.172849,10.275598,3.596091
4,10004,6.883609,3.451677,8.257700,4.257829
...,...,...,...,...,...
9995,19995,4.801310,3.927899,12.143051,3.343776
9996,19996,10.490508,3.265147,7.537821,2.035313
9997,19997,7.491157,3.780172,8.397145,4.202973
9998,19998,12.479392,4.464999,9.365007,3.477778
